###  importing relevant libraries

In [1]:
import warnings
warnings.filterwarnings("ignore")

In [2]:

from tabulate import tabulate
from numpy import array
from random import random
from sklearn import metrics
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis as QDA
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import average_precision_score
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import KFold
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import BernoulliNB#57
from sklearn.naive_bayes import GaussianNB#52
from sklearn.naive_bayes import MultinomialNB#56
from sklearn.naive_bayes import CategoricalNB

from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import Normalizer
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.utils import shuffle
import csv
import math
import matplotlib.pyplot as plt
from collections import Counter
import numpy as np
import os
import pandas as pd
import seaborn as sns
import sklearn
import time
%matplotlib inline


### Discovering Labels

In [3]:
def target_name(name):
    df = pd.read_csv(name,usecols=["Label"])
    target_names=sorted(list(df["Label"].unique()))
    return target_names

In [4]:
def folder(f_name): #this function creates a folder.
    try:
        if not os.path.exists(f_name):
            os.makedirs(f_name)
    except OSError:
        print ("Tthe folder could not be created!")

In [5]:
def find_the_way(path,file_format,con=""):
    files_add = []
    # r=root, d=directories, f = files
    for r, d, f in os.walk(path):
        for file in f:
            if file_format in file:
                if con in file:
                    files_add.append(os.path.join(r, file))  
            
    return files_add

In [6]:
altime=0
#def most_frequent(List): 
#    return max(set(List), key = List.count) 


 
def most_frequent(List):
    occurence_count = Counter(List)
    occurence_count={k: v for k, v in sorted(occurence_count.items(), key=lambda item: item[1],reverse=True)}
    big=list(occurence_count.values())
    big=big.count(big[0])
    return list(occurence_count.keys())[np.random.randint(big)]


def split(a, n):
    k, m = divmod(len(a), n)
    return (a[i * k + min(i, m):(i + 1) * k + min(i + 1, m)] for i in range(n))
def create_exception(df): 
    exception_list=[]
    dominant_mac=[]
    for i in df['aggregated'].unique():
        k=df[df['aggregated']==i]
        for ii in ['MAC']:
            hist = {}
            for x in k[ii].values:
                hist[x] = hist.get(x, 0) + 1
            hist=dict(sorted(hist.items(), key=lambda item: item[1],reverse=True))
            temp=next(iter(hist))
            if temp not in dominant_mac:
                dominant_mac.append(temp)
            else:
                exception_list.append(temp)
    return exception_list




def merged(m_test,predict,step,mixed):
    second=time.time()
    mac_test=[]
    for q in m_test.index:
        mac_test.append(m_test[q])

    d_list=sorted(list(m_test.unique()))
    devices={}
    for q in d_list:
        devices[q]=[]    


    new_y=[0]*len(m_test)

    for q,qq in enumerate (mac_test):
        devices[qq].append(q)
    for q in devices:
        a = [devices[q][j:j + step] for j in range(0, len(devices[q]), step)]  
        for qq in a:
            step_list=[]
            for qqq in qq:
                step_list.append(predict[qqq])
            add=most_frequent(list(step_list))
            for qqq in qq:
                new_y[qqq]=add
    results=pd.DataFrame(m_test)
    results["aggregated"]=new_y
    results["normal"]=predict
    
    #MIXED METHOD
    if mixed:
        exception=create_exception(results)
        for q in exception:
            results.loc[results.MAC == q, 'aggregated'] = results['normal']

    return results["aggregated"].values,time.time()-second

## Calculation of evaluations

In [7]:
def score(altime,train_time,test_time,predict,y_test,class_based_results,i,cv,dname,ml):
    rc=sklearn.metrics.recall_score(y_test, predict,average= "macro")
    pr=sklearn.metrics.precision_score(y_test, predict,average= "macro")
    f_1=sklearn.metrics.f1_score(y_test, predict,average= "macro")     
    accuracy=sklearn.metrics.accuracy_score(y_test, predict)
    accuracy_b=sklearn.metrics.balanced_accuracy_score( y_test,predict)
    kappa=sklearn.metrics.cohen_kappa_score(y_test, predict,labels=None, weights=None, sample_weight=None)
    try:
        roc=sklearn.metrics.roc_auc_score(y_test, predict)
    except:roc=0
    report = sklearn.metrics.classification_report(y_test, predict, target_names=target_names,output_dict=True)
    cr = pd.DataFrame(report).transpose()
    line=[dname,i,cv,ml,accuracy,accuracy_b,pr,rc,f_1,kappa,roc,train_time,test_time,altime]

    if class_based_results.empty:
        class_based_results =cr
    else:
        class_based_results = class_based_results.add(cr, fill_value=0)
    return line,class_based_results

In [8]:
def ML(loop1,loop2,output_csv1,cols,step,mixed,dname):

    #ths = open(output_csv, "w")
    #ths.write("Dataset,T,CV,ML algorithm,Acc,b_Acc,Precision, Recall , F1-score, kappa ,tra-Time,test-Time,Al-Time\n")
    

    from sklearn.metrics import balanced_accuracy_score
    from sklearn.preprocessing import Normalizer
    
    for ii in ml_list:
        #print ('%-15s %-3s %-3s %-6s  %-5s %-5s %-5s %-5s %-8s %-5s %-8s %-8s%-8s%-8s'%
               #("Dataset","T","CV","ML alg","Acc","b_Acc","Prec", "Rec" , "F1", "kap" ,"tra-T","test-T","total","al-time"))
        class_based_results=pd.DataFrame()#"" #pd.DataFrame(0, index=np.arange((len(target_names)+3)), columns=["f1-score","precision","recall","support"])
        cm=pd.DataFrame()
        cv=0
        output_csv=output_csv1.replace("ML",ii)
        repetition=1
        lines=[["Dataset","T","CV","ML","Acc","b_Acc","Prec","Rec","F1","kap","ROC","tra-T","test-T","Al-Time"]]
        for i in range(repetition):



            #TRAIN
            df = pd.read_csv(loop1,usecols=cols)
            m_train=df["MAC"]
            del df["MAC"]
            X_train =df[df.columns[0:-1]]
            X_train=np.array(X_train)
            df[df.columns[-1]] = df[df.columns[-1]].astype('category')
            y_train=df[df.columns[-1]].cat.codes  

            #TEST
            df = pd.read_csv(loop2,usecols=cols)
            df = shuffle(df, random_state=42)
            m_test=df["MAC"]
            del df["MAC"]
            X_test =df[df.columns[0:-1]]
            X_test=np.array(X_test)
            df[df.columns[-1]] = df[df.columns[-1]].astype('category')
            y_test=df[df.columns[-1]].cat.codes




            results_y=[]
            cv+=1
            results_y.append(y_test)


            #machine learning algorithm is applied in this section
            clf = ml_list[ii]#choose algorithm from ml_list dictionary
            second=time.time()
            clf.fit(X_train, y_train)
            train_time=(float((time.time()-second)) )
            second=time.time()
            predict =clf.predict(X_test)
            test_time=(float((time.time()-second)) )
            if step==1:
                altime=0
                line,class_based_results=score(altime,train_time,test_time,predict,y_test,class_based_results,i,cv,dname,ii)
            else:
                predict,altime=merged(m_test,predict,step,mixed)
                line,class_based_results=score(altime,train_time,test_time,predict,y_test,class_based_results,i,cv,dname,ii)
                
                
            lines.append(line)
            df_cm = pd.DataFrame(confusion_matrix(y_test, predict))
            if cm.empty:
                cm =df_cm
            else:
                cm = cm.add(df_cm, fill_value=0)
            
        class_based_results=class_based_results/(repetition)
        results = pd.DataFrame (lines[1:], columns = lines[0])
        results.to_csv(output_csv.replace("ML",ii),index=False)
        results=results.round(3)
        print (tabulate(results, headers=list(results.columns)))
        print()
  
        
        #print (tabulate(class_based_results, headers=list(class_based_results.columns)))
        class_based_results.to_csv(output_csv.replace(".csv","CB.csv"))
        if True:
            cm=cm//repetition
            graph_name=output_csv+"_confusion matrix.pdf"   
            plt.figure(figsize = (40,28))
            sns.heatmap(cm,xticklabels=target_names, yticklabels=target_names, annot=True, fmt='g')
            plt.savefig(graph_name,bbox_inches='tight')#, dpi=400)
            cm.to_csv(graph_name.replace(".pdf",".csv"))
            #plt.show()
            #print(cm)
            #print("\n\n\n")             


            plt.clf()

            
            
            
            
            



# Machine learning applications 

# Aalto Dataset

In [9]:
feature= ['pck_size', 'Ether_type', 'LLC_ctrl', 'EAPOL_version', 'EAPOL_type', 'IP_ihl', 'IP_tos', 'IP_len', 'IP_flags', 'IP_DF', 'IP_ttl', 'IP_options', 'ICMP_code', 'TCP_dataofs', 'TCP_FIN', 'TCP_ACK', 'TCP_window', 'UDP_len', 'DHCP_options', 'BOOTP_hlen', 'BOOTP_flags', 'BOOTP_sname', 'BOOTP_file', 'BOOTP_options', 'DNS_qr', 'DNS_rd', 'DNS_qdcount', 'dport_class', 'payload_bytes', 'entropy',
"MAC",
'Label']


In [10]:
ml_list={"DT" :DecisionTreeClassifier()}



In [11]:
datasets=[['./csvs\\A211102.csv',
  './csvs\\A211105.csv',
  './csvs\\I211102.csv',
  './csvs\\I211103.csv',
  './csvs\\I211104.csv'],
 ['./csvs\\A211108.csv',
  './csvs\\A211109.csv',
  './csvs\\A211115.csv',
  './csvs\\A211116.csv',
  './csvs\\A211117.csv',
  './csvs\\A211118.csv',
  './csvs\\A211122.csv',
  './csvs\\A211123.csv',
  './csvs\\A211125.csv',
  './csvs\\A211206.csv',
  './csvs\\A211208.csv',
  './csvs\\A211223.csv',
  './csvs\\A211225.csv',
  './csvs\\A211228.csv',
  './csvs\\I211108.csv',
  './csvs\\I211112.csv',
  './csvs\\I211115.csv',
  './csvs\\I211118.csv',
  './csvs\\I211119.csv',
  './csvs\\I211122.csv'],
 ['./csvs\\A211110.csv', './csvs\\A211112.csv', './csvs\\I211110.csv'],
 ['./csvs\\A211124.csv',
  './csvs\\I211124.csv',
  './csvs\\I211125.csv',
  './csvs\\I211201.csv',
  './csvs\\I211202.csv',
  './csvs\\I211203.csv',
  './csvs\\I211207.csv',
  './csvs\\I211208.csv',
  './csvs\\I211209.csv',
  './csvs\\I211210.csv',
  './csvs\\I211213.csv',
  './csvs\\I211215.csv',
  './csvs\\I211216.csv',
  './csvs\\I211220.csv',
  './csvs\\I211222.csv',
  './csvs\\I211223.csv'],
 ['./csvs\\A211126.csv', './csvs\\A211207.csv'],
 ['./csvs\\I211116.csv', './csvs\\I211117.csv']]


In [12]:
datasets=[['./csvs\\A211102.csv',
  './csvs\\A211105.csv',
  './csvs\\I211102.csv',
  './csvs\\I211103.csv',
  './csvs\\I211104.csv'],
 ['./csvs\\A211108.csv',
  './csvs\\A211109.csv',
  './csvs\\A211115.csv',
  './csvs\\A211116.csv',
  './csvs\\A211117.csv',
  './csvs\\A211118.csv',
  './csvs\\A211122.csv',
  './csvs\\A211123.csv',
  './csvs\\A211125.csv',
  './csvs\\A211206.csv',
  './csvs\\A211208.csv',
  './csvs\\A211223.csv',
  './csvs\\A211225.csv',
  './csvs\\A211228.csv',
  './csvs\\I211108.csv',
  './csvs\\I211112.csv',
  './csvs\\I211115.csv',
  './csvs\\I211118.csv',
  './csvs\\I211119.csv',
  './csvs\\I211122.csv']]


In [13]:
say=0
dataset="./results/vs/"
folder(dataset)
for no, i in enumerate(datasets):
    for ii in i:
        for iii in i:
            say+=1
            print(f"{say}/698 - {ii[7:-4]} {iii[7:-4]}" )
            train=ii
            test=iii
            mixed=False
            step=1
            sayac=1
            output_csv=f"{dataset}/@{ii[7:-4]}@{iii[7:-4]}.csv"
            target_names=target_name(test)
            ML(train,test,output_csv,feature,step,mixed,dataset[2:-1]+"_"+str(step))   


1/698 - A211102 A211102
    Dataset         T    CV  ML      Acc    b_Acc    Prec    Rec     F1    kap    ROC    tra-T    test-T    Al-Time
--  ------------  ---  ----  ----  -----  -------  ------  -----  -----  -----  -----  -------  --------  ---------
 0  results/vs_1    0     1  DT    0.934    0.905   0.919  0.905  0.909  0.931      0    0.527     0.078          0

2/698 - A211102 A211105
    Dataset         T    CV  ML      Acc    b_Acc    Prec    Rec     F1    kap    ROC    tra-T    test-T    Al-Time
--  ------------  ---  ----  ----  -----  -------  ------  -----  -----  -----  -----  -------  --------  ---------
 0  results/vs_1    0     1  DT    0.889    0.823   0.856  0.823  0.827  0.884      0    1.473      0.17          0

3/698 - A211102 I211102
    Dataset         T    CV  ML      Acc    b_Acc    Prec    Rec     F1    kap    ROC    tra-T    test-T    Al-Time
--  ------------  ---  ----  ----  -----  -------  ------  -----  -----  -----  -----  -------  --------  --------

23/698 - I211104 I211102
    Dataset         T    CV  ML      Acc    b_Acc    Prec    Rec     F1    kap    ROC    tra-T    test-T    Al-Time
--  ------------  ---  ----  ----  -----  -------  ------  -----  -----  -----  -----  -------  --------  ---------
 0  results/vs_1    0     1  DT     0.72    0.826   0.825  0.826  0.795  0.705      0    1.628     0.195          0

24/698 - I211104 I211103
    Dataset         T    CV  ML      Acc    b_Acc    Prec    Rec     F1    kap    ROC    tra-T    test-T    Al-Time
--  ------------  ---  ----  ----  -----  -------  ------  -----  -----  -----  -----  -------  --------  ---------
 0  results/vs_1    0     1  DT    0.896    0.865   0.885  0.865  0.869  0.891      0    1.642     0.177          0

25/698 - I211104 I211104
    Dataset         T    CV  ML      Acc    b_Acc    Prec    Rec     F1    kap    ROC    tra-T    test-T    Al-Time
--  ------------  ---  ----  ----  -----  -------  ------  -----  -----  -----  -----  -------  --------  -----

45/698 - A211108 I211122
    Dataset         T    CV  ML      Acc    b_Acc    Prec    Rec     F1    kap    ROC    tra-T    test-T    Al-Time
--  ------------  ---  ----  ----  -----  -------  ------  -----  -----  -----  -----  -------  --------  ---------
 0  results/vs_1    0     1  DT    0.672    0.579   0.574  0.579  0.533  0.657      0    1.913     0.216          0

46/698 - A211109 A211108
    Dataset         T    CV  ML      Acc    b_Acc    Prec    Rec     F1    kap    ROC    tra-T    test-T    Al-Time
--  ------------  ---  ----  ----  -----  -------  ------  -----  -----  -----  -----  -------  --------  ---------
 0  results/vs_1    0     1  DT    0.921    0.847   0.863  0.847  0.848  0.915      0    1.955     0.295          0

47/698 - A211109 A211109
    Dataset         T    CV  ML      Acc    b_Acc    Prec    Rec     F1    kap    ROC    tra-T    test-T    Al-Time
--  ------------  ---  ----  ----  -----  -------  ------  -----  -----  -----  -----  -------  --------  -----

67/698 - A211115 A211109
    Dataset         T    CV  ML      Acc    b_Acc    Prec    Rec     F1    kap    ROC    tra-T    test-T    Al-Time
--  ------------  ---  ----  ----  -----  -------  ------  -----  -----  -----  -----  -------  --------  ---------
 0  results/vs_1    0     1  DT    0.665     0.61   0.585   0.61  0.566   0.65      0     2.33     0.202          0

68/698 - A211115 A211115
    Dataset         T    CV  ML      Acc    b_Acc    Prec    Rec     F1    kap    ROC    tra-T    test-T    Al-Time
--  ------------  ---  ----  ----  -----  -------  ------  -----  -----  -----  -----  -------  --------  ---------
 0  results/vs_1    0     1  DT    0.946    0.913   0.918  0.913  0.912  0.941      0    2.109     0.236          0

69/698 - A211115 A211116
    Dataset         T    CV  ML      Acc    b_Acc    Prec    Rec     F1    kap    ROC    tra-T    test-T    Al-Time
--  ------------  ---  ----  ----  -----  -------  ------  -----  -----  -----  -----  -------  --------  -----

89/698 - A211116 A211116
    Dataset         T    CV  ML      Acc    b_Acc    Prec    Rec     F1    kap    ROC    tra-T    test-T    Al-Time
--  ------------  ---  ----  ----  -----  -------  ------  -----  -----  -----  -----  -------  --------  ---------
 0  results/vs_1    0     1  DT    0.949     0.92   0.922   0.92  0.918  0.945      0      2.1     0.227          0

90/698 - A211116 A211117
    Dataset         T    CV  ML      Acc    b_Acc    Prec    Rec     F1    kap    ROC    tra-T    test-T    Al-Time
--  ------------  ---  ----  ----  -----  -------  ------  -----  -----  -----  -----  -------  --------  ---------
 0  results/vs_1    0     1  DT    0.759    0.817   0.833  0.817  0.798  0.748      0    1.862     0.203          0

91/698 - A211116 A211118
    Dataset         T    CV  ML      Acc    b_Acc    Prec    Rec     F1    kap    ROC    tra-T    test-T    Al-Time
--  ------------  ---  ----  ----  -----  -------  ------  -----  -----  -----  -----  -------  --------  -----

...

<Figure size 4000x2800 with 0 Axes>

<Figure size 4000x2800 with 0 Axes>

<Figure size 4000x2800 with 0 Axes>

<Figure size 4000x2800 with 0 Axes>

<Figure size 4000x2800 with 0 Axes>

In [14]:
datasets=[ ['./csvs\\A211110.csv', './csvs\\A211112.csv', './csvs\\I211110.csv'],
 ['./csvs\\A211124.csv',
  './csvs\\I211124.csv',
  './csvs\\I211125.csv',
  './csvs\\I211201.csv',
  './csvs\\I211202.csv',
  './csvs\\I211203.csv',
  './csvs\\I211207.csv',
  './csvs\\I211208.csv',
  './csvs\\I211209.csv',
  './csvs\\I211210.csv',
  './csvs\\I211213.csv',
  './csvs\\I211215.csv',
  './csvs\\I211216.csv',
  './csvs\\I211220.csv',
  './csvs\\I211222.csv',
  './csvs\\I211223.csv'],
 ['./csvs\\A211126.csv', './csvs\\A211207.csv'],
 ['./csvs\\I211116.csv', './csvs\\I211117.csv']]


In [15]:
say=0
dataset="./results/vs/"
folder(dataset)
for no, i in enumerate(datasets):
    for ii in i:
        for iii in i:
            say+=1
            print(f"{say}/698 - {ii[7:-4]} {iii[7:-4]}" )
            train=ii
            test=iii
            mixed=False
            step=1
            sayac=1
            output_csv=f"{dataset}/@{ii[7:-4]}@{iii[7:-4]}.csv"
            target_names=target_name(test)
            ML(train,test,output_csv,feature,step,mixed,dataset[2:-1]+"_"+str(step))   


1/698 - A211110 A211110
    Dataset         T    CV  ML      Acc    b_Acc    Prec    Rec     F1    kap    ROC    tra-T    test-T    Al-Time
--  ------------  ---  ----  ----  -----  -------  ------  -----  -----  -----  -----  -------  --------  ---------
 0  results/vs_1    0     1  DT    0.906    0.926   0.928  0.926  0.924  0.901      0    3.212      0.32          0

2/698 - A211110 A211112
    Dataset         T    CV  ML      Acc    b_Acc    Prec    Rec     F1    kap    ROC    tra-T    test-T    Al-Time
--  ------------  ---  ----  ----  -----  -------  ------  -----  -----  -----  -----  -------  --------  ---------
 0  results/vs_1    0     1  DT    0.822    0.823    0.81  0.823  0.799  0.814      0    3.018     0.195          0

3/698 - A211110 I211110
    Dataset         T    CV  ML      Acc    b_Acc    Prec    Rec     F1    kap    ROC    tra-T    test-T    Al-Time
--  ------------  ---  ----  ----  -----  -------  ------  -----  -----  -----  -----  -------  --------  --------

23/698 - A211124 I211220
    Dataset         T    CV  ML      Acc    b_Acc    Prec    Rec     F1    kap    ROC    tra-T    test-T    Al-Time
--  ------------  ---  ----  ----  -----  -------  ------  -----  -----  -----  -----  -------  --------  ---------
 0  results/vs_1    0     1  DT    0.589    0.532   0.516  0.532  0.484  0.573      0    1.255     0.218          0

24/698 - A211124 I211222
    Dataset         T    CV  ML      Acc    b_Acc    Prec    Rec     F1    kap    ROC    tra-T    test-T    Al-Time
--  ------------  ---  ----  ----  -----  -------  ------  -----  -----  -----  -----  -------  --------  ---------
 0  results/vs_1    0     1  DT    0.592    0.541   0.527  0.541  0.494  0.575      0    1.265     0.206          0

25/698 - A211124 I211223
    Dataset         T    CV  ML      Acc    b_Acc    Prec    Rec     F1    kap    ROC    tra-T    test-T    Al-Time
--  ------------  ---  ----  ----  -----  -------  ------  -----  -----  -----  -----  -------  --------  -----

45/698 - I211125 I211201
    Dataset         T    CV  ML      Acc    b_Acc    Prec    Rec     F1    kap    ROC    tra-T    test-T    Al-Time
--  ------------  ---  ----  ----  -----  -------  ------  -----  -----  -----  -----  -------  --------  ---------
 0  results/vs_1    0     1  DT    0.848    0.787   0.844  0.787  0.788  0.842      0    1.659     0.194          0

46/698 - I211125 I211202
    Dataset         T    CV  ML      Acc    b_Acc    Prec    Rec     F1    kap    ROC    tra-T    test-T    Al-Time
--  ------------  ---  ----  ----  -----  -------  ------  -----  -----  -----  -----  -------  --------  ---------
 0  results/vs_1    0     1  DT    0.846    0.783   0.839  0.783  0.791   0.84      0    1.775     0.196          0

47/698 - I211125 I211203
    Dataset         T    CV  ML      Acc    b_Acc    Prec    Rec     F1    kap    ROC    tra-T    test-T    Al-Time
--  ------------  ---  ----  ----  -----  -------  ------  -----  -----  -----  -----  -------  --------  -----

67/698 - I211201 I211210
    Dataset         T    CV  ML      Acc    b_Acc    Prec    Rec     F1    kap    ROC    tra-T    test-T    Al-Time
--  ------------  ---  ----  ----  -----  -------  ------  -----  -----  -----  -----  -------  --------  ---------
 0  results/vs_1    0     1  DT    0.684    0.626   0.621  0.626  0.594  0.672      0    2.017     0.205          0

68/698 - I211201 I211213
    Dataset         T    CV  ML      Acc    b_Acc    Prec    Rec     F1    kap    ROC    tra-T    test-T    Al-Time
--  ------------  ---  ----  ----  -----  -------  ------  -----  -----  -----  -----  -------  --------  ---------
 0  results/vs_1    0     1  DT    0.669    0.595   0.617  0.595  0.571  0.655      0    1.977      0.21          0

69/698 - I211201 I211215
    Dataset         T    CV  ML      Acc    b_Acc    Prec    Rec     F1    kap    ROC    tra-T    test-T    Al-Time
--  ------------  ---  ----  ----  -----  -------  ------  -----  -----  -----  -----  -------  --------  -----

89/698 - I211202 I211223
    Dataset         T    CV  ML      Acc    b_Acc    Prec    Rec     F1    kap    ROC    tra-T    test-T    Al-Time
--  ------------  ---  ----  ----  -----  -------  ------  -----  -----  -----  -----  -------  --------  ---------
 0  results/vs_1    0     1  DT     0.63    0.548   0.549  0.548  0.508  0.615      0    1.717      0.22          0

90/698 - I211203 A211124
    Dataset         T    CV  ML      Acc    b_Acc    Prec    Rec     F1    kap    ROC    tra-T    test-T    Al-Time
--  ------------  ---  ----  ----  -----  -------  ------  -----  -----  -----  -----  -------  --------  ---------
 0  results/vs_1    0     1  DT    0.649    0.584   0.562  0.584  0.552  0.634      0     1.89     0.144          0

91/698 - I211203 I211124
    Dataset         T    CV  ML      Acc    b_Acc    Prec    Rec     F1    kap    ROC    tra-T    test-T    Al-Time
--  ------------  ---  ----  ----  -----  -------  ------  -----  -----  -----  -----  -------  --------  -----

...

<Figure size 4000x2800 with 0 Axes>

<Figure size 4000x2800 with 0 Axes>

<Figure size 4000x2800 with 0 Axes>

<Figure size 4000x2800 with 0 Axes>

<Figure size 4000x2800 with 0 Axes>